# **This notebook documents the step by step process and python codes used in vibe-coding.**
**Goal: Teach the computer to tell emotions from texts**


## Step 1: The setup

We’ll need a library called TextBlob — it’s like a tiny language helper that can “feel” the mood of words.

In [2]:
# install the library (you run this once)
# pip install textblob

from textblob import TextBlob

## Step 2: Give it some sentences to analyze


In [5]:
sentences = [
    "I’m feeling amazing today!",
    "I hate waking up early.",
    "What a beautiful morning!",
    "I’m so tired and bored.",
    "This is the best day ever!"
]

Step 3: Let the AI check the vibe


In [4]:
for text in sentences:
    blob = TextBlob(text)
    mood_score = blob.sentiment.polarity  # ranges from -1 (sad) to +1 (happy)

    if mood_score > 0:
        vibe = "😊 Positive Vibe"
    elif mood_score < 0:
        vibe = "😢 Negative Vibe"
    else:
        vibe = "😐 Neutral Vibe"

    print(f"{text} --> {vibe} (score: {mood_score})")

I’m feeling amazing today! --> 😊 Positive Vibe (score: 0.7500000000000001)
I hate waking up early. --> 😢 Negative Vibe (score: -0.35000000000000003)
What a beautiful morning! --> 😊 Positive Vibe (score: 1.0)
I’m so tired and bored. --> 😢 Negative Vibe (score: -0.45)
This is the best day ever! --> 😊 Positive Vibe (score: 1.0)


What’s happening behind the scenes:

	•	TextBlob looks at the words and how they’re used.
	•	It knows that “amazing,” “beautiful,” and “best” are positive words.
	•	It also knows “hate,” “tired,” and “bored” usually mean negative feelings.
	•	Then it gives each sentence a “vibe score.”

That’s vibe coding in baby form — detecting emotional tone from words.

## To make this vibe detector a little smarter — for example, one that can detect sarcasm or mixed emotions

Now we’re moving from a baby vibe detector → to a teenage vibe detector — one that can spot sarcasm, mixed emotions, and stronger mood swings.


### Step 1: The Problem

Basic vibe coding (like TextBlob) only looks at positive or negative words,
but humans are trickier.

Examples:

“Oh great, another Monday.”
“Yeah, I totally love doing homework.”

Those sound positive on the surface (“great,” “love”)…
but the vibe is clearly sarcastic 😒

So we need an AI that understands context — not just words.

### Step 2: Bring in a Smarter Brain (Transformers 🦾)

Modern AI uses transformer models like BERT, RoBERTa, or DistilBERT — these are like super brains trained on millions of examples of real human speech, tweets, reviews, etc.

They can “feel” tone, detect sarcasm, and sense emotional complexity.

We can use a ready-made one from a library called Hugging Face Transformers.



In [7]:
# install first if needed
# pip install transformers torch

from transformers import pipeline

# load a pre-trained emotion detection model
emotion_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=False)

# try it out!
sentences = [
    "Oh great, another Monday.",
    "I can’t believe I failed again 😔",
    "I’m so proud of myself today!",
    "Sure, because everything always works perfectly (eye roll).",
    "That movie was sad but kind of beautiful."
]

for text in sentences:
    result = emotion_analyzer(text)[0]
    print(f"{text} → {result['label']} ({result['score']:.2f})")

Device set to use cpu


Oh great, another Monday. → joy (0.92)
I can’t believe I failed again 😔 → sadness (0.74)
I’m so proud of myself today! → joy (0.85)
Sure, because everything always works perfectly (eye roll). → neutral (0.82)
That movie was sad but kind of beautiful. → sadness (0.97)


Results aren't exactly what we're expecting because the model we used — j-hartmann/emotion-english-distilroberta-base — is trained mainly on explicit emotions (like joy, sadness, anger, fear, disgust, etc.), not sarcasm.

So when it sees:

“Oh great, another Monday.”
it notices the word “great,” which is usually positive, and doesn’t fully catch the sarcastic tone — because it lacks vocal or situational clues.

Hence:

→ joy (0.92)

The model isn’t “wrong” — it’s doing what it was trained for — but it’s not vibe-aware enough yet.

To make our results more vibe-aware we can do either of the following:
1. Use a model trained specifically for sarcasm
2. Use a multi-label emotion model (some models can say multiple vibes at once - like "sad, but hopeful")
3. Combine Models (Vibe stacking)

Real-world systems (like Spotify or TikTok) often combine:
	•	Emotion model (text emotion)
	•	Sarcasm model
	•	Sentiment model
	•	Context model (user history or audio tone)

Together, that fusion gives a much stronger “vibe sense.”

For the purpose of this exercise we would try out a multi-label emotion model

In [9]:
# install first if needed
# pip install transformers torch

from transformers import pipeline

# load a pre-trained emotion detection model
from transformers import pipeline
sarcasm_detector = pipeline("text-classification", model="mrm8488/t5-base-finetuned-sarcasm-twitter")
sarcasm_detector("Oh great, another Monday.")
# try it out!
sentences = [
    "Oh great, another Monday.",
    "I can’t believe I failed again 😔",
    "I’m so proud of myself today!",
    "Sure, because everything always works perfectly (eye roll).",
    "That movie was sad but kind of beautiful."
]

for text in sentences:
    result = emotion_analyzer(text)[0]
    print(f"{text} → {result['label']} ({result['score']:.2f})")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at mrm8488/t5-base-finetuned-sarcasm-twitter and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu


Oh great, another Monday. → joy (0.99)
I can’t believe I failed again 😔 → sadness (1.00)
I’m so proud of myself today! → joy (1.00)
Sure, because everything always works perfectly (eye roll). → joy (1.00)
That movie was sad but kind of beautiful. → sadness (0.98)


Why sarcasm is hard for AI

Sarcasm needs context, tone, or history — things text alone can’t fully show.

For example:

“Wow, you’re so early…”
If the person actually came late, the vibe is sarcastic. But unless the AI knows that context, it assumes “wow” + “so” + “early” = positive.

That’s why sarcasm detection is an advanced branch of vibe coding. It often needs:
  More context (previous sentences, speaker style)
  Tone of voice (audio input)
  Or models trained on sarcastic datasets (like “Twitter Sarcasm Corpus”)

### **Following this train of thought we would like to see how we could take this same idea and make a music or video vibe detector next, so the AI can feel the mood of a song or clip, not just text.**

### **Step 1: The New Problem — Feeling the Mood of Music & Video**


*Imagine these:*

🎵 *A slow piano ballad → feels melancholy, even if no lyrics.*

🎥 *A fast-cut action trailer with deep bass → feels intense or thrilling.*

🎶 *A pop song with major chords and upbeat tempo → feels joyful.*

🧍 *A TikTok clip where someone is dancing but the music is ironic → mixed vibes.*

Unlike text, here the “vibe” lives in sound + visuals + lyrics.
So, our vibe detector must understand:

🔊 Audio features → tempo, pitch, energy, melody

📝 Lyrics (if present) → using the text vibe detector we built

👁 Visual cues → colors, brightness, facial expressions, motion

### **Step 2: Bring in Audio/Video Brains**

| Modality        | Common Tools / Models                                                                             |
| --------------- | ------------------------------------------------------------------------------------------------- |
| **Audio**       | OpenL3 (audio embeddings), YAMNet (sound classification), Musicnn (genre/mood), Wav2Vec2 (speech) |
| **Lyrics/Text** | Transformers (like before)                                                                        |
| **Visuals**     | CLIP (image+text), OpenCV (basic), or ViT (Vision Transformer) for emotion scenes                 |


We can combine these into a multimodal vibe pipeline

### **Step 3: A Mini Music Vibe Detector (Demo Code)**

Below is a simplified Python example using 🟡 librosa to get audio features + a simple rule-based mood guess.
(You could later swap the “mood detector” with a real model like musicnn.)

📝 This focuses on instrumental audio mood, not lyrics.

In [11]:
# 🎧 Mini Music Vibe Detector (Beginner Demo)
# pip install librosa numpy

import librosa
import numpy as np

def detect_music_vibe(audio_path):
    # 1️⃣ Load the audio file
    y, sr = librosa.load(audio_path, duration=60)  # load first 60 seconds

    # 2️⃣ Extract basic features
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    energy = np.mean(librosa.feature.rms(y=y))
    brightness = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))

    # 3️⃣ Simple vibe classification rules
    if tempo > 120 and energy > 0.05:
        vibe = "🎉 Upbeat / Energetic"
    elif brightness < 2000 and energy < 0.03:
        vibe = "🌧 Sad / Calm / Reflective"
    else:
        vibe = "🎶 Mixed / Neutral Mood"

    return {
        "tempo": round(tempo, 2),
        "energy": float(energy),
        "brightness": float(brightness),
        "detected_vibe": vibe
    }


This little demo extracts tempo, energy, and brightness — super simple audio mood cues.
Later, you can plug in models like:

musicnn
 → pre-trained for music mood/genre.

OpenL3
 → to embed audio → classify moods with a custom model.

### **Step 4: Video Vibe Detector (Concept)**

For videos, you combine:

Audio track → use the music detector above

Transcript (speech) → use a speech-to-text model like Whisper → then feed the text to our teenage vibe detector 🧠

Frames → sample 1 frame per second → run through a Vision Transformer (ViT) or CLIP to detect scene emotion (e.g., bright colors, facial expressions, action).

Mini demo for video frames (concept):

In [12]:
# pip install opencv-python
import cv2

def extract_frames(video_path, frame_rate=1):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = int(fps / frame_rate)
    frames = []
    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if i % frame_interval == 0:
            frames.append(frame)
        i += 1
    cap.release()
    return frames

frames = extract_frames("clip.mp4")
print(f"Extracted {len(frames)} frames for vibe analysis 🖼")


Extracted 0 frames for vibe analysis 🖼


You’d then send these frames into a CLIP model or emotion classifier (e.g., “happy scene,” “tense,” “romantic,” “dark”), and blend that with the audio+text analysis.

### **Step 5: Fuse All the Vibes**

Finally, we combine the scores from:

🎶 Audio mood

📝 Lyrics / Speech vibe

👁 Visual scene vibe

Example (pseudo):

In [14]:
# final_vibe = weighted_average([audio_vibe, text_vibe, visual_vibe])

# This is a conceptual representation of combining different vibe scores.
# In a real implementation, you would define how to combine the scores
# from audio, text, and visual analysis (e.g., using numerical scores and weights).

There are already some multimodal models you can experiment with:

🧠 CLAP (Contrastive Language-Audio Pretraining) → audio + text mood understanding

🧠 VideoCLIP / ViViT → video mood/scene analysis

🧠 AudioSpectrogram Transformers → detect music genre, mood, or emotion directly.

## **In summary:**

Text → Teenage vibe detector 🧠

Music → Adds emotional rhythm and sonic energy 🎧

Video → Adds visual emotion and atmosphere 👁

➕ Fusion = A full sensory vibe intelligence 🌈🤖